## Exercício 1 - POIs e categorias relacionadas a potenciais concorrentes

In [ ]:
import pandas as pd

# Carregando os dados do arquivo
df = pd.read_parquet('pontos_interesse_geograficos.parquet', engine='pyarrow')

# Todos os tipos de negócio únicos
tipos_negocio = df['tipo_negocio'].unique()

# Imprimir a lista de tipos de negócio
print("Lista de tipos de negócio:")
for tipo in tipos_negocio:
    print(tipo)

In [ ]:
import pandas as pd
import geopandas as gpd
import geobr

# Carregando os dados do arquivo 
df = pd.read_parquet('pontos_interesse_geograficos.parquet', engine='pyarrow')

# Filtrando os tipos de negócio que são possíveis concorrentes
tipos_negocio_desejados = ['comercio', 'padaria', 'supermercado']
df_filtrado = df[df['tipo_negocio'].isin(tipos_negocio_desejados)]

# Carregar os dados dos municípios brasileiros usando o geobr
municipios = geobr.read_municipality(code_muni="all", year=2018)

# Join espacial entre os dados filtrados e os dados dos municípios
gdf_filtrado = gpd.GeoDataFrame(df_filtrado, geometry=gpd.points_from_xy(df_filtrado.longitude, df_filtrado.latitude))
gdf_municipios = gpd.GeoDataFrame(municipios)

gdf_join = gpd.sjoin(gdf_filtrado, gdf_municipios, op='within')

# Agrupar por tipo de negócio e município e contar a quantidade de negócios
grupo_tipo_municipio = gdf_join.groupby(['tipo_negocio', 'name_muni']).size().reset_index(name='quantidade')

# Ordenar em ordem decrescente pelo número de negócios
grupo_municipio_quantidade = grupo_tipo_municipio.sort_values('quantidade', ascending=False)

# Exibir o número de negócios por município
for index, row in grupo_municipio_quantidade.iterrows():
    municipio = row['name_muni']
    tipo_negocio = row['tipo_negocio']
    quantidade = row['quantidade']
    print(f"Município: {municipio} | Tipo de Negócio: {tipo_negocio} | Quantidade: {quantidade}")

## Exercício 2 - POIs totais/Município - excluíndo SP

In [ ]:
import pandas as pd
import geopandas as gpd
import geobr

# Carregando os dados do arquivo .parquet usando o pandas
df = pd.read_parquet('pontos_interesse_geograficos.parquet', engine='pyarrow')

# Carregando os dados dos municípios brasileiros usando o geobr
municipios = geobr.read_municipality(code_muni="", year=2018)

# Join espacial entre os dados POIs e os dados dos municípios
gdf_pontos = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf_municipios = gpd.GeoDataFrame(municipios)
gdf_join = gpd.sjoin(gdf_pontos, gdf_municipios, op='within')

# Agrupar por município e contar a quantidade de negócios
grupo_municipio_tipo = gdf_join.groupby(['name_muni', 'tipo_negocio']).size().reset_index(name='quantidade')

# Número total de negócios
grupo_municipio_total = grupo_municipio_tipo.groupby('name_muni')['quantidade'].sum().reset_index(name='total')

# Ordenar em ordem decrescente
grupo_municipio_total = grupo_municipio_total.sort_values('total', ascending=False)

# Exibir o número de negócios por município em ordem decrescente
for index, row in grupo_municipio_total.iterrows():
    municipio = row['name_muni']
    total_negocios = row['total']
    print(f"Município: {municipio} | Total de Negócios: {total_negocios}")

In [ ]:
import pandas as pd
import geopandas as gpd
import geobr
import geopandas.plotting as gplt

# Carregar os dados do arquivo .parquet usando o pandas
df = pd.read_parquet('pontos_interesse_geograficos.parquet', engine='pyarrow')

# Carregar os dados dos municípios brasileiros usando o geobr
municipios = geobr.read_municipality(code_muni="SP", year=2018)

# Realizar um join espacial entre os pontos e os dados dos municípios
gdf_pontos = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf_sp = gpd.sjoin(gdf_pontos, municipios, op='within')

# Agrupar por município e calcular o número total de POIs
grupo_municipio_total = gdf_sp.groupby('name_muni').size().reset_index(name='total')

# Filtrar o município de São Paulo
grupo_municipio_total = grupo_municipio_total[grupo_municipio_total['name_muni'] != 'São Paulo']

# Unir os dados dos municípios de São Paulo com os dados do número total de POIs
municipios_pois = municipios.merge(grupo_municipio_total, on='name_muni')

# Configurar a paleta de cores
cmap = 'Blues'
vmin = municipios_pois['total'].min()
vmax = municipios_pois['total'].max()

# Plotar o mapa
ax = municipios_pois.plot(column='total', figsize=(10,10), cmap=cmap, linewidth=0.8, edgecolor='0.8', legend=True, vmin=vmin, vmax=vmax)


#Adicionar um texto explicativo próximo à escala de cores
ax.annotate('Quantidade de POIs', xy=(1.25, 0.5), xycoords='axes fraction', fontsize=12, ha='left')

# Remover exibição dos eixos
ax.set_axis_off()


## Exercício 3/4 - Mapas das 5 principais categorias e das categorias de possíveis concorrentes

In [ ]:
import pandas as pd

# Carregando os dados do arquivo
df = pd.read_parquet('pontos_interesse_geograficos.parquet', engine='pyarrow')

# Contar a frequência dos tipos de negócio
contagem_tipos_negocio = df['tipo_negocio'].value_counts()

# Selecionar os cinco tipos de negócio mais frequentes
cinco_tipos_negocio_mais_frequentes = contagem_tipos_negocio.head(5)

# Imprimir os resultados
print("Os cinco tipos de negócio mais frequentes:")
print(cinco_tipos_negocio_mais_frequentes)

In [ ]:
import pandas as pd
import geopandas as gpd
import geobr
import geopandas.plotting as gplt

# Carregar os dados do arquivo .parquet usando o pandas
df = pd.read_parquet('pontos_interesse_geograficos.parquet', engine='pyarrow')

# Filtrar os tipos de negócio desejados
categoria = ['padaria','comercio','supermercado'] #variar com o tipo de negocio que quer visualizar
df_filtrado = df[df['tipo_negocio'].isin(categoria)]

# Carregar os dados dos municípios brasileiros usando o geobr
municipios = geobr.read_municipality(code_muni="SP", year=2018)

# Realizar um join espacial entre os pontos e os dados dos municípios
gdf_filtrado = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df_filtrado.longitude, df_filtrado.latitude))
gdf_sp = gpd.sjoin(gdf_filtrado, municipios, op='within')

# Agrupar por município e calcular o número total de POIs
grupo_municipio_total = gdf_sp.groupby('name_muni').size().reset_index(name='total')

# Filtrar o município de São Paulo
grupo_municipio_total = grupo_municipio_total[grupo_municipio_total['name_muni'] != 'São Paulo']

# Unir os dados dos municípios de São Paulo com os dados do número total de POIs
municipios_pois = municipios.merge(grupo_municipio_total, on='name_muni')

# Configurar a paleta de cores
cmap = 'seismic'#alterar a cor também!
vmin = municipios_pois['total'].min()
vmax = municipios_pois['total'].max()

# Plotar o mapa
ax = municipios_pois.plot(column='total', figsize=(10,10), cmap=cmap, linewidth=0.8, edgecolor='0.8', legend=True, vmin=vmin, vmax=vmax)

#Adicionar um texto explicativo próximo à escala de cores
ax.annotate('Qtde de possíveis concorrentes', xy=(1.25, 0.5), xycoords='axes fraction', fontsize=11, ha='left')#alterar a legenda

# Remover exibição dos eixos
ax.set_axis_off()

#Salvar o mapa
municipios_pois.to_file('mapa_possiveis_concorrentes', driver='ESRI Shapefile')#mudar o nome do arquivo!

## Exercício 5 - Correlação espacial da quantidade de POIs por município.

In [ ]:
import pandas as pd
import geopandas as gpd
import geobr
from pysal.lib import weights
from pysal.explore import esda
import numpy as np
from sklearn.preprocessing import StandardScaler

# Carregando os dados do arquivo
df = pd.read_parquet('pontos_interesse_geograficos.parquet')

# Filtrando apenas os pontos no estado de São Paulo
df_sp = df[(df['latitude'] >= -25.5) & (df['latitude'] <= -20.5) & (df['longitude'] >= -54.5) & (df['longitude'] <= -44.0)]

# Carregar os dados dos municípios do estado de São Paulo usando o Geobr
municipios_sp = geobr.read_municipality(code_muni="SP", year=2018)

# Realizar um join espacial entre os dados dos municípios e os pontos de interesse
gdf_sp = gpd.GeoDataFrame(df_sp, geometry=gpd.points_from_xy(df_sp.longitude, df_sp.latitude))
gdf_sp = gpd.sjoin(gdf_sp, municipios_sp, op='within')

# Agrupar por município e calcular a quantidade total de pontos de interesse
grupo_municipio_total = gdf_sp.groupby('code_muni').size().reset_index(name='total')

# Merge entre os dados dos municípios e a quantidade total de pontos de interesse
municipios_pois = municipios_sp.merge(grupo_municipio_total, on='code_muni')

# Criar a matriz de pesos espaciais com base na distância
w = weights.KNN.from_dataframe(municipios_pois, k=3) #Mudar esse peso aqui, tentei K, Rook e Queen.

# Padronizar a variável total
scaler = StandardScaler()
municipios_pois['total_standardized'] = scaler.fit_transform(municipios_pois[['total']])

# Calcular o coeficiente I de Moran global
moran_global = esda.Moran(municipios_pois['total_standardized'].values, w)
moran_global_summary = moran_global.I

# Imprimir o resultado
print("Coeficiente I de Moran Global:", moran_global_summary)

<h3> k= 4  - Coeficiente I de Moran Global: 0.19 </h3>
<h3> k = 5 - Coeficiente I de Moran Global: 0.19 </h3>
<h3> Rook - Coeficiente I de Moran Global: 0.31 </h3>
<h3> Queen - Coeficiente I de Moran Global: 0.33 </h3>


## Exercício 6 - Plotar os dados de faturamento, exibindo informações no ponto.

In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from geopy.geocoders import Nominatim

# Carregar o arquivo parquet
df = pd.read_parquet('unidades_faturamento.parquet')

# Calcular a mediana do faturamento para cada unidade
df['mediana_faturamento'] = df.iloc[:, 1:10].median(axis=1)

# Função para formatar o valor em reais
def formatar_reais(valor):
    valor_str = f'{valor:,.2f}'
    valor_str = valor_str.replace('.', '#').replace(',', '.').replace('#', ',')
    return f'R$ {valor_str}'

# Converter a coluna de faturamento para o formato em reais
df['mediana_faturamento'] = df['mediana_faturamento'].apply(formatar_reais)

# Converter as colunas de latitude e longitude para coordenadas geográficas
geometry = gpd.points_from_xy(df['longitude'], df['latitude'])
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Plotar os dados no mapa usando Plotly
fig = px.scatter_mapbox(gdf, lat='latitude', lon='longitude', hover_data=['latitude', 'longitude', 'mediana_faturamento'],
                        hover_name='id_unidade',
                        mapbox_style='open-street-map', zoom=5.5)

fig.update_layout(title='Unidades')
fig.show()


## Exercício 7 - Colocar no mapa tendência do faturamento

In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from sklearn.linear_model import LinearRegression
from geopy.geocoders import Nominatim

# Carregar o arquivo parquet
df = pd.read_parquet('unidades_faturamento.parquet')

# Calcular a mediana do faturamento para cada unidade
df['mediana_faturamento'] = df.iloc[:, 1:10].median(axis=1)

# Calcular a tendência de alta ou baixa do faturamento usando regressão linear
X = df.iloc[:, 1:10].values
y = df['mediana_faturamento'].values
reg = LinearRegression().fit(X, y)

# Prever a tendência para cada unidade
df['tendencia'] = reg.predict(X) - y
df['tendencia'] = df['tendencia'].apply(lambda x: 'Alta' if x > 0 else 'Baixa')

geolocator = Nominatim(user_agent='my_geocoder')

def obter_municipio(latitude, longitude):
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    if location:
        address = location.raw['address']
        municipio = address.get('city', '')
        return municipio
    return ''

df['municipio'] = df.apply(lambda row: obter_municipio(row['latitude'], row['longitude']), axis=1)

# Calcular a tendência média por município
tendencia_media = df.groupby('municipio')['tendencia'].apply(lambda x: (x == 'Alta').mean())

# Filtrar os municípios com tendência alta e baixa
municipios_alta = tendencia_media[tendencia_media > 0.5].index.tolist()
municipios_baixa = tendencia_media[tendencia_media <= 0.5].index.tolist()

# Imprimir a lista de municípios com tendência alta
print('Municípios com tendência alta:')
for municipio in municipios_alta:
    print(municipio)

# Imprimir a lista de municípios com tendência baixa
print('Municípios com tendência baixa:')
for municipio in municipios_baixa:
    print(municipio)

In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from sklearn.linear_model import LinearRegression

# Carregar o arquivo parquet
df = pd.read_parquet('unidades_faturamento.parquet')

# Calcular a mediana do faturamento para cada unidade
df['mediana_faturamento'] = df.iloc[:, 1:10].median(axis=1)


# Calcular a tendência de alta ou baixa do faturamento usando regressão linear
X = df.iloc[:, 1:10].values
y = df['mediana_faturamento'].values
reg = LinearRegression().fit(X, y)

# Prever a tendência para cada unidade
df['tendencia'] = reg.predict(X)-y
df['tendencia'] = df['tendencia'].apply(lambda x: 'Alta' if x > 0 else 'Baixa')

# Função para formatar o valor em reais
def formatar_reais(valor):
    valor_str = f'{valor:,.2f}'
    valor_str = valor_str.replace('.', '#').replace(',', '.').replace('#', ',')
    return f'R$ {valor_str}'

# Converter a coluna de faturamento para o formato em reais
df['mediana_faturamento'] = df['mediana_faturamento'].apply(formatar_reais)


# Converter as colunas de latitude e longitude para coordenadas geográficas
geometry = gpd.points_from_xy(df['longitude'], df['latitude'])
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Plotar os dados no mapa usando Plotly
fig = px.scatter_mapbox(gdf, lat='latitude', lon='longitude', hover_data=['latitude', 'longitude', 'mediana_faturamento', 'tendencia'],
                        hover_name='id_unidade',
                        mapbox_style='open-street-map', zoom=5.5)

fig.update_layout(title='Faturamento das unidades')
fig.show()

## Exercício 8 - Mapa com faturamento por município

In [ ]:
import pandas as pd
import geopandas as gpd
import geobr

# Carregar o arquivo parquet
df = pd.read_parquet('unidades_faturamento.parquet')

# Carregar os dados dos municípios de SP usando o geobr
municipios = geobr.read_municipality(code_muni="SP", year=2020)

# Criar um DataFrame com todos os municípios de São Paulo
municipios_sp = municipios[['name_muni', 'geometry']]

# Realizar um join espacial entre as unidades e os dados dos municípios
gdf_unidades = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf_sp = gpd.sjoin(gdf_unidades, municipios_sp, op='within')

# Agrupar por município e calcular o faturamento total por mês
grupo_municipio_faturamento = gdf_sp.groupby('name_muni').sum().iloc[:, 1:11]
num_unidades = gdf_sp.groupby('name_muni').size()
grupo_municipio_faturamento['faturamento_medio'] = grupo_municipio_faturamento.sum(axis=1) / (num_unidades * 10)

# Unir os dados dos municípios de São Paulo com os dados do faturamento médio
municipios_faturamento = municipios_sp.merge(grupo_municipio_faturamento['faturamento_medio'], left_on='name_muni', right_index=True, how='left')

# Preencher os valores de faturamento médio faltantes com zero
municipios_faturamento['faturamento_medio'].fillna(0, inplace=True)

# Configurar a paleta de cores
cmap = 'Blues'
vmin = municipios_faturamento['faturamento_medio'].min()
vmax = municipios_faturamento['faturamento_medio'].max()

# Plotar o mapa
ax = municipios_faturamento.plot(column='faturamento_medio', figsize=(10, 10), cmap=cmap, linewidth=0.8, edgecolor='0.8', legend=True, vmin=vmin, vmax=vmax)

# Adicionar um texto explicativo próximo à escala de cores
ax.annotate('Faturamento Médio (em 10 milhões de reais)', xy=(1.25, 0.5), xycoords='axes fraction', fontsize=12, ha='left')

# Remover exibição dos eixos
ax.set_axis_off()

# Ordenar o DataFrame pelo faturamento médio em ordem decrescente
top_10_faturamentos = municipios_faturamento.sort_values(by='faturamento_medio', ascending=False).head(10)

# Imprimir os 10 maiores faturamentos
print(top_10_faturamentos)


## Exercício 10 - Correlação espacial de faturamento total por município.


In [ ]:
import pandas as pd
import geopandas as gpd
import geobr
from pysal.lib import weights
from pysal.explore import esda
import numpy as np
from sklearn.preprocessing import StandardScaler

# Carregar o arquivo parquet
df = pd.read_parquet('unidades_faturamento.parquet')

# Carregar os dados dos municípios de SP usando o geobr
municipios = geobr.read_municipality(code_muni="SP", year=2020)

# Criar um DataFrame com todos os municípios de São Paulo
municipios_sp = municipios[['name_muni', 'geometry']]

# Realizar um join espacial entre as unidades e os dados dos municípios
gdf_unidades = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf_sp = gpd.sjoin(gdf_unidades, municipios_sp, op='within')

# Agrupar por município e calcular o faturamento total por mês
grupo_municipio_faturamento = gdf_sp.groupby('name_muni').sum().iloc[:, 1:11]
grupo_municipio_faturamento['faturamento_total'] = grupo_municipio_faturamento.sum(axis=1)

# Unir os dados dos municípios de São Paulo com os dados do faturamento médio
municipios_faturamento = municipios_sp.merge(grupo_municipio_faturamento['faturamento_total'], left_on='name_muni', right_index=True, how='left')

# Preencher os valores de faturamento médio faltantes com zero
municipios_faturamento['faturamento_total'].fillna(0, inplace=True)

# Criar a matriz de pesos espaciais com base na distância
w = weights.KNN.from_dataframe(municipios_faturamento, k=5) #Mudar esse peso aqui, tentei K, Rook e Queen.

# Padronizar a variável total
scaler = StandardScaler()
municipios_faturamento['total_standardized'] = scaler.fit_transform(municipios_faturamento[['faturamento_total']])

# Calcular o coeficiente I de Moran global
moran_global = esda.Moran(municipios_faturamento['faturamento_total'].values, w)
moran_global_summary = moran_global.I

# Imprimir o resultado
print("Coeficiente I de Moran Global:", moran_global_summary)

<h3> k= 4  - Coeficiente I de Moran Global: 0.18 </h3>
<h3> k = 5 - Coeficiente I de Moran Global: 0.17 </h3>
<h3> Rook - Coeficiente I de Moran Global: 0.25 </h3>
<h3> Queen - Coeficiente I de Moran Global: 0.26 </h3>
